In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import io

# Load model
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

def load_image(uploaded_file):
    img = Image.open(uploaded_file).convert('RGB')
    img = img.resize((512, 512))
    return tf.image.convert_image_dtype(np.array(img), tf.float32)[tf.newaxis, ...]

from google.colab import files

# Upload content and style images
print("Upload your content image (e.g., a photo):")
content_file = files.upload()
content_key = list(content_file.keys())[0]
content_img = load_image(io.BytesIO(content_file[content_key]))

print("\nUpload your style image (e.g., a painting):")
style_file = files.upload()
style_key = list(style_file.keys())[0]
style_img = load_image(io.BytesIO(style_file[style_key]))

# Style transfer
stylized_img = model(tf.constant(content_img), tf.constant(style_img))[0]
stylized_img = tf.squeeze(stylized_img)

# Display result
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(content_img[0])
plt.title('Content Image')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(stylized_img)
plt.title('Stylized Image')
plt.axis('off')
plt.show()


In [ ]:
!pip install streamlit tensorflow tensorflow_hub pillow pyngrok



In [8]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
import numpy as np

model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

def load_image(uploaded_file):
    img = Image.open(uploaded_file).convert('RGB')
    img = img.resize((512, 512))
    return tf.image.convert_image_dtype(np.array(img), tf.float32)[tf.newaxis, ...]

st.title("Neural Style Transfer Web App")

content_file = st.file_uploader("Upload your content image", type=["jpg", "png", "jpeg"])
style_file = st.file_uploader("Upload your style image", type=["jpg", "png", "jpeg"])

if content_file and style_file:
    content_img = load_image(content_file)
    style_img = load_image(style_file)
    stylized_img = model(tf.constant(content_img), tf.constant(style_img))[0]
    stylized_img = tf.squeeze(stylized_img)
    st.image([content_img[0].numpy(), stylized_img.numpy()], caption=['Content Image', 'Stylized Image'], width=300)


Overwriting app.py


In [9]:
!streamlit run app.py --server.port 8501 &>/dev/null &



In [10]:
from pyngrok import ngrok

# Set your ngrok authtoken (use your actual token)
ngrok.set_auth_token("2zEi1fOnKLD7QAkapKybJFnJKBD_5Ks4P5GiZ8Q9KGsg214c7")

# Start ngrok tunnel to port 8501 (Streamlit default is 8501)
public_url = ngrok.connect(addr=8501, proto="http")  # <-- Use addr and proto as keyword arguments
print("Your public URL is:", public_url.public_url)


Your public URL is: https://6907-34-74-230-209.ngrok-free.app
